In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('/content/model111.h5')


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
from PIL import Image
import io

# Function to preprocess the image
def preprocess_image(img):
    img = img.resize((150, 150))  # Resize to match model input
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Load an image file
def predict(image_path):
    # Load and preprocess the image
    img = Image.open(image_path)
    img_array = preprocess_image(img)

    # Make a prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    # Map class indices to labels
    class_labels = {0: 'Apple', 1: 'Banana', 2: 'Orange', 3: 'Strawberry', 4: 'Grapes'}  # Update with your actual labels
    predicted_label = class_labels.get(predicted_class[0], "Unknown")

    return predicted_label

# Example usage
image_path = '/content/pexels-pixabay-206959.jpg'  # Replace with the path to the image you want to classify
prediction = predict(image_path)
print(f'Predicted class: {prediction}')


1/1 [==============================] - 1s 1s/step
Predicted class: Apple


In [ ]:
!pip install --upgrade gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

# Load the model
model = load_model('/content/model111.h5')

# Define the function for making predictions
def predict(img):
    img = img.resize((150, 150))  # Resize image to match model input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    # Map class indices to labels
    class_labels = {0: 'Apple', 1: 'Banana', 2: 'Orange', 3: 'Strawberry', 4: 'Grapes'}  # Replace with your actual class labels
    predicted_label = class_labels.get(predicted_class[0], "Unknown")

    return predicted_label

# Define the Gradio interface
interface = gr.Interface(
    fn=predict,  # The function to call for predictions
    inputs=gr.Image(type="pil"),  # Define the input type (image) with PIL format
    outputs=gr.Text(),  # Define the output type (text)
    live=True  # Update the interface live as the user interacts
)

# Launch the interface
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b21ce1084bdcdeefae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install fastapi uvicorn
!pip install fastapi uvicorn pyngrok


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
from PIL import Image
import io

app = FastAPI()

# Load the model
model = load_model('/content/model111.h5')  # Update the path to your model

# Define the function for making predictions
def preprocess_image(img):
    img = img.resize((150, 150))  # Resize image to match model input
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)
    return img_array

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Read the image file
    image_data = await file.read()
    img = Image.open(io.BytesIO(image_data))

    # Preprocess the image and make a prediction
    img_array = preprocess_image(img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    # Map class indices to labels
    class_labels = {0: 'Apple', 1: 'Banana', 2: 'Orange', 3: 'Strawberry', 4: 'Grapes'}  # Replace with your actual class labels
    predicted_label = class_labels.get(predicted_class[0], "Unknown")

    return JSONResponse(content={"prediction": predicted_label})

# Run the app with `uvicorn` from the command line:
# uvicorn app:app --reload


In [ ]:
# Install required packages
!pip install fastapi uvicorn pyngrok tensorflow pillow

# Import libraries
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
from PIL import Image

# Apply the asyncio patch
nest_asyncio.apply()

# Define FastAPI app
app = FastAPI()

# Load your trained model
model = load_model('/content/model111.h5')

# Class labels for your model (replace with your actual labels)
class_labels = {0: 'Apple', 1: 'Banana', 2: 'Orange', 3: 'Strawberry', 4: 'Grapes'}

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/predict/")
async def predict(img_file: UploadFile = File(...)):
    # Read the image file
    img = Image.open(img_file.file).convert('RGB')
    img = img.resize((150, 150))  # Resize image to match model input

    # Preprocess the image
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_label = class_labels.get(predicted_class, "Unknown")

    return {"prediction": predicted_label}

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
# and paste it below
NGROK_AUTH_TOKEN = "2ke5PGVAO8SWvcI86S3z3KVbJbW_prnk9StBHKCj8H1MeBDd"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Set up ngrok to tunnel the server
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the server
uvicorn.run(app, host="0.0.0.0", port=8000)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-24' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

Public URL: NgrokTunnel: "https://4037-34-27-224-169.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [422]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     39.58.243.248:0 - "GET / HTTP/1.1" 200 OK
INFO:     39.58.243.248:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     39.58.243.248:0 - "GET /openapi.json HTTP/1.1" 200 OK
1/1 [==============================] - 1s 940ms/step
INFO:     39.58.243.248:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     39.58.243.248:0 - "GET / HTTP/1.1" 200 OK
INFO:     39.58.243.248:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     39.58.243.248:0 - "GET /openapi.json HTTP/1.1" 200 OK
1/1 [==============================] - 0s 88ms/step
INFO:     39.58.243.248:0 - "POST /predict/ HTTP/1.1" 200 OK
